In [ ]:
from typing import Callable

import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erfc

%matplotlib ipympl

# Session 3: Multi-antenna channels and techniques

In the first session of this project, you were introduced to the application of point processes and Monte-Carlo techniques for evaluating a user's network performance. In the second session, you have studied two models for the shadowing: either using a ray-tracing-like model, or with a stochastic model. 

This third session first focuses on the small-scale fading. Then, building on this, multi-antenna networks will be considered. This raises the challenge of i) modelling the MIMO channels, and ii) exploiting the MIMO channel.   

## Fading modelling

Considering a wireless communication, the instantaneous received power can be computed as 
 $$ P_R =P_T \, L(d)\, s \,|h|^2, $$
 with $P_R$ and $P_T$ the received and transmitted power (considering unit antenna gains), $L(d)$ the path loss which depends on the communication distance $d$, $s$ the shadowing power gain, and $|h|^2$ the fading power gain.
 
As in the previous sessions, the path loss will be modelled with
$$ L(d) = \frac{1}{\left(\sqrt{h^2+d^2}\right)^{\alpha}}, $$
with $h$ the BS height and $\alpha$ the path loss exponent. The shadowing will be left aside for this session, as the focus is on the fading phenomenon. 
 
The fading power gain $|h|^2$ is a random variable for which several distributions can be considered. The simplest one is the Rayleigh model, in which
- $h$ is a unit-variance zero-mean complex normal random variable;
- $|h|$ follows thus a Rayleigh distribution, whose PDF is given by $f_{|h|}(x) = 2xe^{-x^2}$ for $x\geq 0$;
- $|h|^2$ follows thus an exponential distibution, whose PDF is given by $f_{|h|^2}(x) = e^{-x}$ for $x\geq 0$. 

Other models (such as Nagakami fading, or Ricean fading) can be considered. Yet, they are all such that $\mathbb{E}\left[\left|h\right|^2\right] =1$ since the average power gain of the fading should be equal to $1$, by definition.

To begin this session, **implement** i) the path loss model function and ii) the SISO fading model function.

In [ ]:
# TODO: Implement this function


def path_loss_model(r: np.ndarray, h: float = 1.0, alpha: float = 3.0) -> np.ndarray:
    """
    Returns the path loss at a distance r from the BS, with a height h, and with a path loss exponent alpha.
    """
    L = np.zeros_like(r)  # Change me
    # :START-REMOVE:
    L = (h**2 + r**2) ** (-alpha / 2)
    # :END-REMOVE:
    return L

In [ ]:
# TODO: Implement this function


def SISO_fading() -> float:
    """
    Returns the fading coefficient $h$, with as modelthe Rayleigh fading model
    """
    h = 0.0  # Change me
    # :START-REMOVE:
    h = (np.random.randn() + 1j * np.random.randn()) / np.sqrt(2)
    # :END-REMOVE:
    return h

The below cell computes the average power gain of the fading (which should be equal to 1). Verify that you have correctly normalized your fading coefficient.

In [ ]:
# Check that the Rayleigh variables have been correctly normalised.
N_draw = 4000
realisations = np.zeros(N_draw)

for n in range(N_draw):
    fading = SISO_fading()
    realisations[n] = np.square(np.abs(fading))

# Draw how the average converges to 1
increasing_av = np.cumsum(realisations) / np.arange(1, N_draw + 1)

plt.figure()
plt.plot(increasing_av, "b")
plt.grid()
plt.show()

To observe the impact of the fading, the received power a user gets from its BS can be studied, **with** and **without** fading. To that aim, **implement the two below functions**, the first one samples points using a PPP. The second one gets the realizations of the received power, with and without fading. 

In [ ]:
# TODO: Implement this function


def sample_PPP(lam: float = 0.03, L: float = 10.0) -> np.ndarray:
    """
    Samples points in a square of side L, using a PPP with parameter lam.
    """
    points = np.random.rand(10, 2)  # Change me
    # :START-REMOVE:
    N = np.random.poisson(lam * L**2)
    points_init = np.random.rand(N, 2)
    points = (points_init - 0.5) * L
    # :END-REMOVE:
    return points

In [ ]:
# TODO: Implement this function


def draws_power_Rayleigh(
    lam_BS: float = 0.03,
    P0: float = 10.0,
    h: float = 1.0,
    alpha: float = 3.0,
    N_geo: int = 2000,
    L: float = 30.0,
):
    """
    Computes the average power received by  at typical UE located at (0, 0) over N_geo draws, without or with Rayleigh fading
    """
    power_array_no_fading = np.zeros((N_geo,))
    power_array_fading = np.zeros((N_geo,))
    # :START-REMOVE:
    for m in range(N_geo):
        points_BS = sample_PPP(lam=lam_BS, L=L)
        distance = np.sqrt((points_BS[:, 0]) ** 2 + (points_BS[:, 1]) ** 2)
        d_min = np.min(distance)  # closest BS

        path_loss = path_loss_model(d_min, h, alpha)
        fading = SISO_fading()
        P_R_no_fading = P0 * path_loss
        P_R_fading = P_R_no_fading * np.abs(fading) ** 2
        power_array_no_fading[m] = P_R_no_fading
        power_array_fading[m] = P_R_fading

    # :END-REMOVE:
    return (power_array_no_fading, power_array_fading)

With the above functions, **plot the power realizations** and **compute the average received power** for $P_0=10$, $h=1$, $\alpha = 3$, $\lambda_{\text{BS}} \in [0.004,0.4]$, $L=100$, and $N_{\text{geo}}=2000$, with and without fading. Save the power realisations in the arrays `results_no_fading` and `results_fading`.

What is the impact of the fading on the average receive power?  

In [ ]:
# TODO: Complete this cell accordingly


# :START-REMOVE:
alpha = 3.0
N_geo = 20000
P0 = 10
h = 1
L = 100
lambda_vec = np.linspace(0.004, 0.4, 30)
N_lam = len(lambda_vec)

results_no_fading = np.zeros((N_geo, N_lam))
results_fading = np.zeros((N_geo, N_lam))
for n in range(N_lam):
    lam = lambda_vec[n]
    (results_no_fading[:, n], results_fading[:, n]) = draws_power_Rayleigh(
        lam_BS=lam, P0=P0, h=h, alpha=alpha, N_geo=N_geo, L=L
    )

plt.figure()
(line,) = plt.plot(
    lambda_vec,
    10 * np.log10(np.mean(results_no_fading, axis=0)),
    "-+",
    label="No fading",
)
plt.plot(
    lambda_vec,
    10 * np.log10(np.mean(results_fading, axis=0)),
    color=line.get_color(),
    linestyle="--",
    label="Fading",
)

plt.legend()
plt.xlabel("BS density [BS/unit area]")
plt.ylabel("Average received power [dB]")
plt.show()
# :END-REMOVE:

As always, not only the mean quantity is of interest, but the whole distribution of the received power must be studied. **Obtain the CCDF of the received power** in dB scale, with and without fading, for a density of your choice. 

What do you observe? How can you explain it? 

In [ ]:
# TODO: Complete this cell accordingly

# :START-REMOVE:

lambda_target = 0.04
idx_lam = np.argmin(
    np.abs(lambda_target - lambda_vec)
)  # idx of the dsneity closest to the targetted one


powers_ppp_no_fading = results_no_fading[:, idx_lam]
hist_ppp_no_fading, bin_edges_ppp_no_fading = np.histogram(
    10 * np.log10(powers_ppp_no_fading), bins="auto", density=True
)
CDF_ppp_no_fading = np.cumsum(hist_ppp_no_fading * np.diff(bin_edges_ppp_no_fading))
CCDF_ppp_no_fading = 1 - CDF_ppp_no_fading
(line,) = plt.plot(
    bin_edges_ppp_no_fading[:-1], CCDF_ppp_no_fading, "-", label="No fading"
)

powers_ppp_fading = results_fading[:, idx_lam]
hist_ppp_fading, bin_edges_ppp_fading = np.histogram(
    10 * np.log10(powers_ppp_fading), bins="auto", density=True
)
CDF_ppp_fading = np.cumsum(hist_ppp_fading * np.diff(bin_edges_ppp_fading))
CCDF_ppp_fading = 1 - CDF_ppp_fading
(line,) = plt.plot(
    bin_edges_ppp_fading[:-1],
    CCDF_ppp_fading,
    color=line.get_color(),
    linestyle="--",
    label="Fading",
)
plt.legend()

plt.xlabel("Received power [dB]")
plt.grid()
plt.show()
# :END-REMOVE:

While the distribution of the received power is interesting, it is difficult to extract from it the user performance. Typically, such performance would be the average bit error rate. 

**Implement the below function**, which computes the BER corresponding to a given SNR. You can choose a modulation scheme of your choice.  

In [ ]:
# TODO: Implement this function


def BER(
    SNR: float = 10,
) -> float:
    """
    Returns the BER corresponding to the SNR, for a communication scheme which can be chosen by the user
    """
    ber = 0.0  # Change me
    # :START-REMOVE:
    ber = 0.5 * erfc(np.sqrt(SNR))  # For BPSK
    # :END-REMOVE:
    return ber

We are now able to compute the average BER a user gets. To that aim, two methods will be employed.

1. Given a noise power $\sigma^2_n$, and given the power realizations, the realizations of the SNR and of the BER can be computed. Then, the average BER can be obtained. 
2. Given a noise power $\sigma^2_n$, and given the PDF of the received power in dB $\left[P_R\right]_{\text{dB}}$ given by $f_Y(y)$,  the average BER can be obtained as 
$$ \overline{\text{BER}} = \int_{-\infty}^{\infty} \text{BER}\left(\frac{10^{\frac{y}{10}}}{\sigma^2_n}\right)f_Y(y)\mathrm{d}y,$$
where $\text{BER}(\cdot)$ is the instantaneous BER function. This integral is computed numerically, based on the estimate of the PDF given by the vectors `bin_edges_ppp` and `hist_ppp`. 

**Draw a BER vs SNR curve** for the two methods, with and without fading, for $\sigma^2_n \in \left[10^{-2},0\right]$. From the power distribution and from the noise powr, you should compute the average SNR for each noise power and use this as x-axis.

What do you observe? What are the pros and cons of the methods ? 

**Prove that the above formula indeed gives the average BER**. 

In [ ]:
# TODO: Complete this cell accordingly


# :START-REMOVE:
N_SNR = 50
SNR_vec = np.logspace(0, 2, N_SNR)
P_mean = (np.mean(powers_ppp_no_fading) + np.mean(powers_ppp_fading)) / 2
sigma_n_vec = P_mean / SNR_vec

p_e_vec_no_fading = np.zeros(N_SNR)
p_e_vec_fading = np.zeros(N_SNR)
p_e_vec_no_fading_distr = np.zeros(N_SNR)
p_e_vec_fading_distr = np.zeros(N_SNR)

for n in range(N_SNR):
    sigma_sq_n = sigma_n_vec[n]

    # method 1
    SNR_no_fading_real = powers_ppp_no_fading / sigma_sq_n
    p_e_vec_no_fading[n] = np.mean(BER(SNR_no_fading_real))

    SNR_fading_real = powers_ppp_fading / sigma_sq_n
    p_e_vec_fading[n] = np.mean(BER(SNR_fading_real))

    # method 2
    PR_no_fading_dB = (bin_edges_ppp_no_fading[1:] + bin_edges_ppp_no_fading[:-1]) / 2
    PR_fading_dB = (bin_edges_ppp_fading[1:] + bin_edges_ppp_fading[:-1]) / 2

    p_e_vec_no_fading_distr[n] = np.trapz(
        0.5
        * erfc(np.sqrt((10 ** (PR_no_fading_dB / 10)) / sigma_sq_n))
        * hist_ppp_no_fading,
        PR_no_fading_dB,
    )
    p_e_vec_fading_distr[n] = np.trapz(
        0.5 * erfc(np.sqrt((10 ** (PR_fading_dB / 10)) / sigma_sq_n)) * hist_ppp_fading,
        PR_fading_dB,
    )


plt.figure()
plt.semilogy(10 * np.log10(SNR_vec), p_e_vec_no_fading, "-", label="No fading")
plt.semilogy(
    10 * np.log10(SNR_vec),
    p_e_vec_no_fading_distr,
    "-",
    label="No fading (from distr)",
)
plt.semilogy(10 * np.log10(SNR_vec), p_e_vec_fading, "--", label="Fading")
plt.semilogy(
    10 * np.log10(SNR_vec), p_e_vec_fading_distr, "--", label="Fading (from distr)"
)
plt.grid()
plt.legend()
plt.show()
# :END-REMOVE:

Another metric of interest, which will be needed later on, is the outage probability. This probability is the probability that the channel capacity is smaller than a targetted rate $R$:
$$ p_\text{out} \triangleq \mathbb{P}\left[\log\left(1+\text{SNR}\right) \leq R\right],$$
with the instantaneous SNR given by $\text{SNR}\triangleq \frac{P_R}{\sigma^2_n}$. The point in studying this quantity is that, at high SNR, it shows the same behaviour than the BER curve. The linear decay of the error probability in the case of Rayleigh fading can therefore also be observed from the outage probability. What is particularly interesting is that the outage probability can directly be obtained from the received power distribution, and thus no integration is needed in this case.

> Letting $\chi \triangleq 10\log_{10}\left(\frac{\text{SNR}}{\overline{\text{SNR}}}\right) = 10\log_{10}\left(\frac{P_R}{\overline{P_R}}\right)$, and letting $F_{\chi}(x)$ be the CDF of $\chi$, **prove that** 
$$p_\text{out} = F_{\chi}\left(K -\left[\overline{\text{SNR}}\right]_{\text{dB}}\right),$$
with $K$ a constant depending on the targetted rate.

In the below cell, **plot the coverage probability** as a function of $K -\left[\overline{\text{SNR}}\right]_{\text{dB}}$, directly from the CDF of the received power. Make sure that the Y-axis is in log scale. Without loss of generality, you can consider $K=0$.

In [ ]:
# TODO: Complete this cell accordingly

# :START-REMOVE:

mean_power_no_fading = np.mean(results_no_fading[:, idx_lam])
hist_chi_no_fading = hist_ppp_no_fading
bins_chi_no_fading = bin_edges_ppp_no_fading - 10 * np.log10(mean_power_no_fading)
CDF_chi_no_fading = np.cumsum(hist_chi_no_fading * np.diff(bins_chi_no_fading))

(line,) = plt.semilogy(
    -bins_chi_no_fading[:-1], CDF_chi_no_fading, "-", label="No fading"
)

mean_power_fading = np.mean(results_fading[:, idx_lam])
hist_chi_fading = hist_ppp_fading
bins_chi_fading = bin_edges_ppp_fading - 10 * np.log10(mean_power_no_fading)
CDF_chi_fading = np.cumsum(hist_chi_fading * np.diff(bins_chi_fading))

(line,) = plt.semilogy(-bins_chi_fading[:-1], CDF_chi_fading, "-", label="Fading")

plt.legend()
plt.xlabel("$ [SNR]_{dB}-K$")
plt.ylabel("Outage probability")
plt.grid()
plt.show()
# :END-REMOVE:

## MIMO channel modelling

Now that single-antenna fading has been covered, MIMO channels are studied. Let us consider an emitter equipped with $n_T$ antennas and a receiver with $n_R$ antennas. The channel can be described by a channel matrix $\tilde{\mathbf{H}} \in \mathbb{C}^{n_R \times n_T}$, whose coefficient $\tilde{h}_{ij}$ is the complex channel coefficient between transmitter antenna $i$ and receiver antenna $j$. Such channel coefficient can include the path loss, shadowing, and fading. To begin, the shadowing will not be considered.

The horizontal distance $d$ between the emitter and the receiver is usually much greater than the dimension of the antenna arrays, and thus the path loss can be considered constant across all antenna pairs. Therefore, the channel model becomes
$$ \tilde{\mathbf{H}} = L(d) \mathbf{H},$$
with $L(d)$ the path loss function and $\mathbf{H}$ the fading matrix.

Regarding the fading, many random models can be considered to model $\mathbf{H}$. All these models must nevertheless be such that $\mathbb{E}\left[\left|h_{ij}\right|^2\right] =1$ whatever $i$ and $j$, by definition of the fading. In this session, two basic channel models will be considered.
   > **The Rayleigh i.i.d. channel model**: each entry $h_{ij}$ is modelled as a unit-variance complex normal random variable, independent from the other coefficients.
   
   > **The fully correlated Rayleigh model**: All $h_{ij}$ are equal to the same coefficient $h$, with $h$ being a unit-variance complex normal random variable.

To begin the MIMO analysis, **implement** i) the MIMO i.i.d. channel model, and ii) the MIMO fully correlated channel model.

In [ ]:
# TODO: Implement this function

ChannelModel = Callable[[int, int], np.ndarray]


def MIMO_channel_iid(n_T: int, n_R: int) -> np.ndarray:
    """
    Returns the fading channel matrix between a transmitter equipped with n_T antennas and a receiver equipped with n_R antennas,
    with a Rayleigh i.i.d. channel model
    """
    H = np.zeros((n_R, n_T))
    # :START-REMOVE:
    H = (np.random.randn(n_R, n_T) + 1j * np.random.randn(n_R, n_T)) / np.sqrt(
        2
    )  # divided by sqrt of 2 factor to have a unit variance
    # :END-REMOVE:
    return H

In [ ]:
# TODO: Implement this function


def MIMO_channel_full_cor(n_T: int, n_R: int) -> np.ndarray:
    """
    Returns the fading channel matrix between a transmitter equipped with n_T antennas and a receiver equipped with n_R antennas,
    with a fully correlated Rayleigh channel model
    """
    H = np.zeros((n_R, n_T))
    # :START-REMOVE:
    h = (np.random.randn() + 1j * np.random.randn()) / np.sqrt(2)
    H = h * np.ones((n_R, n_T))
    # :END-REMOVE:
    return H

It is important to ensure the channel matrix elements have correctly be normalised to have a in average a unit power gain. **Implement in the below cell a test** proving this.

In [ ]:
# TODO: Complete this cell accordingly

# :START-REMOVE:


# Check that the Rayleigh variables have been correctly normalised.
N_draw = 4000
n_T = 2
n_R = 2
realisations_iid = np.zeros(N_draw * n_T * n_R)
realisations_full_cor = np.zeros(N_draw * n_T * n_R)

for n in range(N_draw):
    channel_iid = MIMO_channel_iid(n_T, n_R)
    realisations_iid[n * n_T * n_R : (n + 1) * n_T * n_R] = np.ndarray.flatten(
        np.square(np.abs(channel_iid))
    )

    channel_full_cor = MIMO_channel_full_cor(n_T, n_R)
    realisations_full_cor[n * n_T * n_R : (n + 1) * n_T * n_R] = np.ndarray.flatten(
        np.square(np.abs(channel_full_cor))
    )

# Draw how the average converges to 1
increasing_av_iid = np.cumsum(realisations_iid) / np.arange(1, N_draw * n_T * n_R + 1)
increasing_av_full_cor = np.cumsum(realisations_full_cor) / np.arange(
    1, N_draw * n_T * n_R + 1
)
plt.figure()
plt.plot(increasing_av_iid, "b", label="i.i.d. Rayleigh")
plt.plot(increasing_av_full_cor, "r", label="Fully correlated Rayleigh")
plt.grid()
plt.legend()
plt.show()
# :END-REMOVE:

To conclude this part of the session, it should be emphasized that, between the two extreme cases of i.i.d. entries, and fully correlated entries, many models exist. Some of these models have been covered during the lecture. 

## MIMO techniques

Thanks to the use of multiple antennas at the emitter and at the receiver, the communication can be improved. Two main directions can be considered for such improvement. Either a unique communication stream is considered between one emitter and one receiver, and the multiple antennes enable to improve the S(I)NR of the transmission. This is what will be referred to as **beamforming** techniques. When both the emitter and the receiver are equipped with multiple antennas, then it is also possible to send multiple communication streams simultaneously, on the different antennas. This process is named **multiplexing**. In this session, only the beamforming techniques will be considered. 

With beamforming techniques, mathematically, in order to transmit the signal $x(t)$, an amplitude and a phase shift coefficient is applied on each of the antennas. These complex coefficients are grouped into the vector $\mathbf{w}_T \in \mathbb{C}^{n_T\times 1}$, and the transmitted signals read thus as $\mathbf{x}_{T}(t) = \mathbf{w}_Tx(t)$. At the receiver antennas, the signal is obtained as $\mathbf{x}_{R}(t) = \tilde{\mathbf{H}}\mathbf{x}_{T}(t) + \mathbf{n}(t)  = \tilde{\mathbf{H}}\mathbf{w}_Tx(t) + \mathbf{n}(t)$, with $\mathbf{n}(t)$ the noise vector whose elements are almost always assumed to be i.i.d. AWGN noises. Then, a beamforming vector $\mathbf{w}_R \in \mathbb{C}^{1 \times n_R}$ is applied, in order to obtain $y(t) = \mathbf{w}_R\mathbf{x}_{R}(t)$, leading to 
$$y(t)  = \mathbf{w}_R\tilde{\mathbf{H}}\mathbf{w}_Tx(t) + \mathbf{w}_R\mathbf{n}(t).$$ 

Based on this expression, the signal power is obtained as 
$$P_s = \left|\mathbf{w}_R\tilde{\mathbf{H}}\mathbf{w}_T\right|^2 P_0,$$
and the noise power as
$$ P_N = \|\mathbf{w}_R\|^2_2 \sigma^2_n,$$
with $P_0$ and $\sigma^2_n$ the signal and noise powers. Note that the interference can be computed in a similar way. 

**Implement the two below functions**, computing these powers based on the beamformers and the channel matrix `H`. 

In [ ]:
# TODO: Implement this function


def received_power(
    path_loss: np.ndarray, H: np.ndarray, w_T: np.ndarray, w_R: np.ndarray, P_0: float
) -> np.ndarray:
    """
    Returns the receiver power, based on the path loss, channel matrix,
    transmit and receive beamformers w_T and w_R, and the power P_0 of signal.
    """
    P_R = 0.0  # Change me
    # :START-REMOVE:
    Power_coeff_beamforming = (
        np.abs(w_R @ H @ w_T) ** 2
    )  # power coeffient due to the beamforming
    P_R = P_0 * path_loss * Power_coeff_beamforming
    # :END-REMOVE:
    return P_R

In [ ]:
# TODO: Implement this function


def noise_power(w_R: np.ndarray, sigma_sq_n: float) -> float:
    """
    Returns the noise power after the receiver beamforming.
    """
    P_N = 0.0
    # :START-REMOVE:
    P_N = np.linalg.norm(w_R, 2) ** 2 * sigma_sq_n
    # :END-REMOVE:
    return P_N

As you can observe, the norm of the beamforming vectors impacts the transmit and receive power. Hence, such beamforming vectors are assumed to be normalized in order to have a unit power: $\|\mathbf{w}_T\|^2_2=1$, $\|\mathbf{w}_R\|^2_2=1$.

Such vectors can be designed to optimize the system performances. Several possibilities are presented in the lectures, in the SIMO, MISO and MIMO cases. Let us focus first on MISO techniques, and more precisely on the Maximum Ratio Transmission (MRT) technique which optimizes the SNR. **Implement the below function**, providing the transmit beamforming vector based on the channel matrix. 

In [ ]:
# TODO: Implement this function


def MRT(H):
    """
    Returns the MRT vector that maximises the SNR of a given
    channel matrix (shape 1 x n_T).
    """
    # Channel_Mat is the vector containing the channel coefficients, it should be of shape 1xn_T
    # return w = h^H/||h||, a n_T \times 1 vector
    w = np.zeros_like(H).T  # Change me
    # :START-REMOVE:
    w = H.conj().T / np.linalg.norm(H, 2)
    # :END-REMOVE:
    return w

## Monte-Carlo analysis of the received power 

Now that the MIMO channel models have been presented, and that the beamforming techniques have been implemented, the performance of a user can be evaluated with Monte-Carlo methods.

To begin, we are interested in the received power a single-antenna user gets from the BS, when the latter has $n_T$ antennas and performs a MRT beamforming.

**Implement the below function**, getting the realizations of the received power when MRT beamforming is employed. 

In [ ]:
# TODO: Implement this function


def draws_power_with_beamforming(
    MIMO_Channel_model: ChannelModel = MIMO_channel_iid,
    lam_BS: float = 0.03,
    P0: float = 10.0,
    h: float = 1.0,
    alpha: float = 3.0,
    N_geo: int = 2000,
    L: float = 30.0,
    n_T: int = 1,
    n_R: int = 1,
):
    """
    Computes the average power received by  at typical UE located at (0, 0)
    over N_geo draws, when the BS performs the beamforming.
    The fading channel is given by the MIMO_Channel_model function.
    """
    power_array = np.zeros((N_geo,))
    # :START-REMOVE:
    for m in range(N_geo):
        points_BS = sample_PPP(lam=lam_BS, L=L)
        distance = np.sqrt((points_BS[:, 0]) ** 2 + (points_BS[:, 1]) ** 2)
        d_min = np.min(distance)  # closest BS

        path_loss = path_loss_model(d_min, h, alpha)
        Channel_Mat = MIMO_Channel_model(n_T, n_R)
        w_T = MRT(Channel_Mat)
        w_R = np.array([[1]])
        power_array[m] = received_power(path_loss, Channel_Mat, w_T, w_R, P0)

    # :END-REMOVE:
    return power_array

**Draw samples of the received powers**, considering the i.i.d. channel model and the fully correlated one. Conider BS densities ranging from $\lambda_{\text{BS}} = 0.004,..., 0.4$, and number of antennas $n_T = 1,2,5,20$.  

In [ ]:
# TODO: Complete this cell accordingly


MRT_results_iid = []
MRT_results_full_cor = []

# :START-REMOVE:
alpha = 3.0
N_geo = 20000
P0 = 10
h = 1
L = 100
lambda_vec = np.linspace(0.004, 0.4, 30)
N_lam = len(lambda_vec)
N_T_vec = np.array([1, 2, 5, 20])

for n_T in N_T_vec:
    row_iid = np.zeros((N_geo, N_lam))
    row_full_cor = np.zeros((N_geo, N_lam))
    for n in range(N_lam):
        lam = lambda_vec[n]
        row_iid[:, n] = draws_power_with_beamforming(
            MIMO_channel_iid,
            lam_BS=lam,
            P0=P0,
            h=h,
            alpha=alpha,
            N_geo=N_geo,
            L=L,
            n_T=n_T,
            n_R=1,
        )

        row_full_cor[:, n] = draws_power_with_beamforming(
            MIMO_channel_full_cor,
            lam_BS=lam,
            P0=P0,
            h=h,
            alpha=alpha,
            N_geo=N_geo,
            L=L,
            n_T=n_T,
            n_R=1,
        )

    MRT_results_iid.append(row_iid)
    MRT_results_full_cor.append(row_full_cor)
# :END-REMOVE:

Based on the above draws, **compare the averaged received powers** as a function of the BS densities, for the various number of antennas and the two channel model. Are you able to observe the array gain, and the diversity gain? 

In [ ]:
# TODO: Complete this cell accordingly

# :START-REMOVE:
plt.figure()
# Square results
for i, n_T in enumerate(N_T_vec):
    (line,) = plt.plot(
        lambda_vec,
        10 * np.log10(np.mean(MRT_results_iid[i], axis=0)),
        "-",
        label=f"(i.i.d.) - $n_T = {n_T}$",
    )
    plt.plot(
        lambda_vec,
        10 * np.log10(np.mean(MRT_results_full_cor[i], axis=0)),
        color=line.get_color(),
        linestyle="--",
        label=f"(Fully correlated) - $n_T = {n_T}$",
    )

plt.legend()
plt.xlabel("BS density [BS/unit area]")
plt.ylabel("Average received power [dB]")
plt.show()
# :END-REMOVE:

Not only the average power matters, but the power distribution is also important. Based on the above draws, **draw the CCDF of the received power**, for the various number of antennas and the two channel models. Are you able to observe the array gain, and the diversity gain? You can consider any BS density $\lambda_{\text{BS}}$. 

In [ ]:
# TODO: Complete this cell accordingly

# :START-REMOVE:
lambda_target = 0.04
idx_lam = np.argmin(
    np.abs(lambda_target - lambda_vec)
)  # idx of the density closest to the targetted one

for i, n_T in enumerate(N_T_vec):
    # i.i.d. channel
    powers_ppp = MRT_results_iid[i][:, idx_lam]
    hist_ppp, bin_edges_ppp = np.histogram(
        10 * np.log10(powers_ppp), bins="auto", density=True
    )
    CDF_ppp = np.cumsum(hist_ppp * np.diff(bin_edges_ppp))
    CCDF_ppp = 1 - CDF_ppp
    (line,) = plt.plot(
        bin_edges_ppp[:-1], CCDF_ppp, "-", label=f"(i.i.d.) - $n_T = {n_T}$"
    )

    # fully correlated channel
    powers_ppp = MRT_results_full_cor[i][:, idx_lam]
    hist_ppp, bin_edges_ppp = np.histogram(
        10 * np.log10(powers_ppp), bins="auto", density=True
    )
    CDF_ppp = np.cumsum(hist_ppp * np.diff(bin_edges_ppp))
    CCDF_ppp = 1 - CDF_ppp
    (line,) = plt.plot(
        bin_edges_ppp[:-1],
        CCDF_ppp,
        color=line.get_color(),
        linestyle="--",
        label=f"(Fully correlated) - $n_T = {n_T}$",
    )


plt.legend()


plt.xlabel("Received power [dB]")
plt.grid()
plt.show()
# :END-REMOVE:

If you have implemented everything correctly, you should be able to quantify the array gain in the above figure. You should also be able to observe qualitatively the diversity gains, but it is difficult to quantify it.

To that aim, the average BER/SNR cruve can be drawn. **Implement the below cell** to obtain such figure. You can choose to implement either the average of the BER realisations method, or the numerical integration method.

In [ ]:
# TODO: Complete this cell accordingly

# :START-REMOVE:
N_SNR = 50
SNR_vec = np.logspace(0, 3, N_SNR)
p_e_iid = np.zeros((len(N_T_vec), N_SNR))
p_e_full_cor = np.zeros((len(N_T_vec), N_SNR))

for i, n_T in enumerate(N_T_vec):
    P_mean = (
        np.mean(MRT_results_iid[i][:, idx_lam])
        + np.mean(MRT_results_full_cor[i][:, idx_lam])
    ) / 2
    sigma_n_vec = P_mean / SNR_vec

    for n in range(N_SNR):
        sigma_sq_n = sigma_n_vec[n]

        SNR_real_iid = MRT_results_iid[i][:, idx_lam] / sigma_sq_n
        p_e_iid[i, n] = np.mean(BER(SNR_real_iid))

        SNR_real_full_cor = MRT_results_full_cor[i][:, idx_lam] / sigma_sq_n
        p_e_full_cor[i, n] = np.mean(BER(SNR_real_full_cor))

for i, n_T in enumerate(N_T_vec):
    (line,) = plt.semilogy(
        10 * np.log10(SNR_vec), p_e_iid[i, :], "-", label=f"(i.i.d.) - $n_T = {n_T}$"
    )
    (line,) = plt.semilogy(
        10 * np.log10(SNR_vec),
        p_e_full_cor[i, :],
        color=line.get_color(),
        linestyle="--",
        label=f"(Fully correlated) - $n_T = {n_T}$",
    )

plt.legend()
plt.grid()
plt.xlabel("Mean SNR [dB]")
plt.ylabel("Average BER")
plt.show()
# :END-REMOVE:

While some linear tendency can be observed in the above figure, it is difficult to draw conclusions from this. One can also look at the outage probability. 

In [ ]:
# TODO: Complete this cell accordingly

# :START-REMOVE:
for i, n_T in enumerate(N_T_vec):
    # i.i.d. channel
    powers_ppp = MRT_results_iid[i][:, idx_lam]
    mean_power_ppp = np.mean(powers_ppp)
    hist_ppp, bin_edges_ppp = np.histogram(
        10 * np.log10(powers_ppp), bins="auto", density=True
    )
    hist_chi = hist_ppp
    bins_chi = bin_edges_ppp - 10 * np.log10(mean_power_ppp)
    CDF_chi = np.cumsum(hist_chi * np.diff(bins_chi))

    (line,) = plt.plot(
        -bins_chi[:-1], 10 * np.log10(CDF_chi), "-", label=f"(i.i.d.) - $n_T = {n_T}$"
    )

    # find diversity gain
    x_coord = -bins_chi[:-1]
    y_coord = 10 * np.log10(CDF_chi)
    y_min = 10 * np.log10(10 / N_geo)
    y_max = y_min + 20
    idx_lin = np.where((y_min <= y_coord) & (y_coord <= y_max))
    p = np.polyfit(x_coord[idx_lin], y_coord[idx_lin], 1)
    # plot linear fit
    plt.plot(
        x_coord[idx_lin], np.polyval(p, x_coord[idx_lin]), "+", color=line.get_color()
    )
    # print diversity gain
    print(f"diversity gain - $n_T = {n_T}$, i.i.d. : {-p[0]}")

    # fully correlated channel
    powers_ppp = MRT_results_full_cor[i][:, idx_lam]
    mean_power_ppp = np.mean(powers_ppp)
    hist_ppp, bin_edges_ppp = np.histogram(
        10 * np.log10(powers_ppp), bins="auto", density=True
    )
    hist_chi = hist_ppp
    bins_chi = bin_edges_ppp - 10 * np.log10(mean_power_ppp)
    CDF_chi = np.cumsum(hist_chi * np.diff(bins_chi))

    (line,) = plt.plot(
        -bins_chi[:-1],
        10 * np.log10(CDF_chi),
        color=line.get_color(),
        linestyle="--",
        label=f"(Fully correlated) - $n_T = {n_T}$",
    )

    # find diversity gain
    x_coord = -bins_chi[:-1]
    y_coord = 10 * np.log10(CDF_chi)
    y_min = 10 * np.log10(10 / N_geo)
    y_max = y_min + 20
    idx_lin = np.where((y_min <= y_coord) & (y_coord <= y_max))
    p = np.polyfit(x_coord[idx_lin], y_coord[idx_lin], 1)
    # plot linear fit
    plt.plot(
        x_coord[idx_lin], np.polyval(p, x_coord[idx_lin]), "+", color=line.get_color()
    )
    # print diversity gain
    print(f"diversity gain - $n_T = {n_T}$, full cor. : {-p[0]}")


plt.legend()
plt.xlabel("$ [SNR]_{dB}-K$")
plt.ylabel("Outage probability [dB]")
plt.grid()
plt.show()
# :END-REMOVE:

From the above, **compare** the numerical and the theoretical diversity gains. What can you conclude? You may need to increase the amount of samples in order to observe the linear decays.

## Monte-Carlo analysis of the SINR

The received power, in itself, is not sufficient to characterize the communication quality. One can instead **assess the SINR of the user**. 

As a result, one must evaluate the power received from the other BS, that also performs beamforming towards another user. To that aim, one must define which beamformer the other BSs implement. One possibility is to consider that each BS serves a user, and thus has a beamformer adapted to the fading matrix.  For each BS, such channel matrix can be drawn randomly and the beamforming vectors of all BS can be computed. Then, the SINR can be evaluated.

In [ ]:
# TODO: Implement this function


def draws_SINR_with_beamforming(
    MIMO_Channel_model: ChannelModel = MIMO_channel_iid,
    lam_BS: float = 0.03,
    P0: float = 10.0,
    h: float = 1.0,
    alpha: float = 3.0,
    N_geo: int = 2000,
    L: float = 30.0,
    n_T: int = 1,
    n_R: int = 1,
    sigma_sq_n: float = 1.0,
):
    """
    Computes the average interference received by
    at typical UE located at (0, 0) over N_geo draws.
    """
    SINR_array = np.zeros((N_geo,))
    # :START-REMOVE:
    for m in range(N_geo):
        points_BS = sample_PPP(lam=lam_BS, L=L)
        N_BS = len(points_BS)
        distance = np.sqrt((points_BS[:, 0]) ** 2 + (points_BS[:, 1]) ** 2)
        i_closest = np.argmin(distance)  # closest BS

        P_int = 0
        # for each BS, draw a random channel matrix, implement MRT, and compute power
        for n in range(N_BS):
            if n != i_closest:
                Channel_Mat_other_user = MIMO_Channel_model(n_T, n_R)
                w_T_other_user = MRT(Channel_Mat_other_user)
                Channel_Mat_typical_user = MIMO_Channel_model(
                    n_T, n_R
                )  # between interfering BS and typical user
                Channel_Mat_typical_user /= np.linalg.norm(Channel_Mat_typical_user)
                path_loss_typical_user = path_loss_model(distance[n], h, alpha)
                P_int += received_power(
                    path_loss_typical_user,
                    Channel_Mat_typical_user,
                    w_T_other_user,
                    np.array([[1]]),
                    P0,
                )

        # useful link
        Channel_use = MIMO_Channel_model(n_T, n_R)
        w_T_use = MRT(Channel_use)
        path_loss_use = path_loss_model(distance[i_closest], h, alpha)
        P_use = received_power(path_loss_use, Channel_use, w_T_use, np.array([[1]]), P0)

        SINR_array[m] = P_use / (P_int + sigma_sq_n)
    # :END-REMOVE:
    return SINR_array

In [ ]:
# TODO: Complete this cell accordingly


MRT_results_SINR = []
# :START-REMOVE:
# power distribution
alpha = 3.0
N_geo = 1000
P0 = 10
h = 1
L = 100
lam = 0.04
N_T_vec = np.array([1, 2, 5, 20])
sigma_sq_n = 0.02
for n_T in N_T_vec:
    row = draws_SINR_with_beamforming(
        MIMO_channel_iid,
        lam_BS=lam,
        P0=P0,
        h=h,
        alpha=alpha,
        N_geo=N_geo,
        L=L,
        n_T=n_T,
        n_R=1,
        sigma_sq_n=sigma_sq_n,
    )

    MRT_results_SINR.append(row)

for i, n_T in enumerate(N_T_vec):
    powers_ppp = MRT_results_SINR[i]
    hist_ppp, bin_edges_ppp = np.histogram(
        10 * np.log10(powers_ppp), bins="auto", density=True
    )
    CDF_ppp = np.cumsum(hist_ppp * np.diff(bin_edges_ppp))
    CCDF_ppp = 1 - CDF_ppp
    plt.plot(bin_edges_ppp[:-1], CCDF_ppp, "+-", label=f"PPP - $n_T = {n_T}$")
plt.legend()
plt.xlabel("SINR [dB]")
plt.grid()
plt.show()
# :END-REMOVE:

Another possibility would be to do the following. First, generate two PPPs, one for the BSs, and one for the UEs. For each BS, determine which UEs are associated with it, and select one of them randomly. For the BSs which do not have any user associated with them, consider they do not emit anything and therefore that they can be removed from the PPP. With this geometry, compute the interference, SINR, or other metrics. 